In [ ]:
# ==========================================
# STEP 1: ENVIRONMENT SETUP & GPU CHECK
# ==========================================

# Check GPU availability
!nvidia-smi

import torch
print(f"\n✅ PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected - training will be SLOW on CPU")

In [ ]:
# ==========================================
# STEP 2: INSTALL REQUIRED PACKAGES
# ==========================================

!pip install -q yfinance pandas-ta xgboost scikit-learn joblib python-dotenv requests fredapi

print("\n✅ Packages installed successfully")
print("   - yfinance: Historical OHLCV data")
print("   - pandas-ta: Technical indicators")
print("   - xgboost: GPU-accelerated gradient boosting")
print("   - scikit-learn: Random Forest, Gradient Boosting, preprocessing")
print("   - fredapi: FRED economic data (VIX, yields)")

## Step 3: Mount Google Drive (Save Models Here)

In [ ]:
# ==========================================
# STEP 3: MOUNT GOOGLE DRIVE
# ==========================================

from google.colab import drive
drive.mount('/content/drive')

import os

# Create workspace structure
workspace = '/content/drive/MyDrive/underdog_trader'
os.makedirs(workspace, exist_ok=True)
os.makedirs(f'{workspace}/models', exist_ok=True)
os.makedirs(f'{workspace}/data', exist_ok=True)
os.makedirs(f'{workspace}/results', exist_ok=True)

print(f"\n✅ Workspace created: {workspace}")
print(f"   - models/: Trained model files (.pkl)")
print(f"   - data/: Cached OHLCV data (.parquet)")
print(f"   - results/: Training metrics (.json)")

## Step 4: Clone Repository & Import Modules

In [ ]:
# ==========================================
# STEP 4: CLONE QUANTUM-AI-TRADER REPO
# ==========================================

# Clone repository (contains our custom modules)
!git clone https://github.com/alexpayne556-collab/quantum-ai-trader_v1.1.git /content/quantum-ai-trader

# Add to Python path
import sys
sys.path.append('/content/quantum-ai-trader/src/python')

print("✅ Repository cloned successfully")

In [ ]:
# ==========================================
# STEP 5: IMPORT CUSTOM MODULES
# ==========================================

from multi_model_ensemble import MultiModelEnsemble
from feature_engine import FeatureEngine
from regime_classifier import RegimeClassifier

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import logging
import joblib
import json

logging.basicConfig(level=logging.INFO)
print("✅ Modules imported successfully")

## Step 5: Define Alpha 76 Watchlist + Kodiak AI Clarification

In [ ]:
# ==========================================
# STEP 6: ALPHA 76 WATCHLIST
# ==========================================

# Alpha 76 high-velocity small/mid-cap stocks
# NOTE: KDK = Kodiak Gas Services (energy infrastructure, NOT Kodiak Robotics/AI)
ALPHA_76 = [
    # Autonomous & AI Hardware (15 tickers)
    'SYM', 'IONQ', 'RGTI', 'QUBT', 'AMBA', 'LAZR', 'INVZ', 'OUST', 'AEVA', 'SERV',
    'MBLY', 'TSLA', 'KDK', 'AI', 'SOUN',
    
    # Space Economy (12 tickers)
    'RKLB', 'ASTS', 'LUNR', 'JOBY', 'ACHR', 'PL', 'SPIR', 'IRDM', 'SPCE', 'RDW', 'MAXR', 'IRTC',
    
    # Biotech (16 tickers)
    'VKTX', 'NTLA', 'BEAM', 'CRSP', 'EDIT', 'VERV', 'BLUE', 'FATE', 'AKRO', 'KOD',
    'CYTK', 'LEGN', 'RARE', 'SRPT', 'BMRN', 'ALNY',
    
    # Green Energy & Grid (9 tickers)
    'FLNC', 'NXT', 'BE', 'ARRY', 'ENPH', 'ENOV', 'QS', 'VST', 'AES',
    
    # Fintech & Digital Assets (9 tickers)
    'SOFI', 'COIN', 'HOOD', 'UPST', 'AFRM', 'LC', 'MARA', 'SQ', 'NU',
    
    # Next-Gen Consumer & Software (15 tickers)
    'APP', 'DUOL', 'PATH', 'S', 'CELH', 'ONON', 'SOUN', 'FOUR', 'NET', 'GTLB',
    'DDOG', 'SNOW', 'PLTR', 'RBLX', 'U'
]

print(f"✅ Alpha 76 Watchlist: {len(ALPHA_76)} tickers")
print(f"   Sectors: Autonomous, Space, Biotech, Energy, Fintech, Software")
print(f"\n   NOTE: KDK = Kodiak Gas Services (energy infrastructure for AI data centers)")
print(f"         NOT Kodiak Robotics/AI (which is PRIVATE, not publicly traded)")

## Step 6: Download Historical Data (2 Years, 1hr Bars)

In [ ]:
# ==========================================
# STEP 7: DOWNLOAD OHLCV DATA
# ==========================================

def download_alpha76_data(tickers: list, period: str = '2y', interval: str = '1h') -> pd.DataFrame:
    """
    Download OHLCV data for all tickers
    
    Expected failures: 5-10 tickers (delisted, insufficient history)
    Expected success: 65-70 tickers with ~3,500 bars each = ~250k total rows
    """
    print(f"\n🚜 Harvesting {len(tickers)} tickers...")
    print(f"   Period: {period} | Interval: {interval}")
    print(f"   Expected time: 10-15 minutes\n")
    
    all_data = []
    failed_tickers = []
    
    for i, ticker in enumerate(tickers, 1):
        try:
            # Download data
            df = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)
            
            if df.empty or len(df) < 200:
                failed_tickers.append(ticker)
                print(f"   [{i}/{len(tickers)}] ⚠️ {ticker}: Insufficient data ({len(df)} bars)")
                continue
            
            # Add ticker column
            df['ticker'] = ticker
            df['timestamp'] = df.index
            df.reset_index(drop=True, inplace=True)
            
            all_data.append(df)
            print(f"   [{i}/{len(tickers)}] ✅ {ticker}: {len(df)} bars")
            
        except Exception as e:
            failed_tickers.append(ticker)
            print(f"   [{i}/{len(tickers)}] ❌ {ticker}: {e}")
    
    # Combine all data
    result = pd.concat(all_data, ignore_index=True)
    
    print(f"\n{'='*60}")
    print(f"✅ Download Complete")
    print(f"   Success: {len(tickers) - len(failed_tickers)} tickers")
    print(f"   Failed: {len(failed_tickers)} tickers {failed_tickers if failed_tickers else ''}")
    print(f"   Total rows: {len(result):,}")
    print(f"{'='*60}\n")
    
    return result

# Execute download
raw_data = download_alpha76_data(ALPHA_76)

# Save to cache
cache_path = f'{workspace}/data/alpha76_raw_data.parquet'
raw_data.to_parquet(cache_path)
print(f"✅ Data cached to: {cache_path}")

## Step 7: Feature Engineering (49 Technical Indicators)

In [ ]:
# ==========================================
# STEP 8: CALCULATE FEATURES
# ==========================================

# Initialize feature engine
feature_engine = FeatureEngine()

print("🧠 Engineering features for all tickers...")
print(f"   Expected time: 5-10 minutes\n")

# Group by ticker and calculate features
featured_data = []

for ticker in raw_data['ticker'].unique():
    ticker_df = raw_data[raw_data['ticker'] == ticker].copy()
    
    try:
        # Calculate 49 technical indicators
        features_df = feature_engine.calculate_all_features(ticker_df)
        
        # Calculate MAX EXCURSION target (not just close-to-close)
        # BUY: Price hits +3% at ANY point in next 5 bars
        indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=5)
        features_df['fwd_max_high'] = features_df['High'].rolling(window=indexer).max()
        features_df['fwd_min_low'] = features_df['Low'].rolling(window=indexer).min()
        
        # Target labels
        buy_threshold = 0.03  # +3%
        sell_threshold = -0.02  # -2%
        
        features_df['fwd_max_return'] = (features_df['fwd_max_high'] - features_df['Close']) / features_df['Close']
        features_df['fwd_min_return'] = (features_df['fwd_min_low'] - features_df['Close']) / features_df['Close']
        
        # Labels: BUY=1, SELL=-1, HOLD=0
        features_df['target'] = 0  # Default: HOLD
        features_df.loc[features_df['fwd_max_return'] >= buy_threshold, 'target'] = 1  # BUY
        features_df.loc[features_df['fwd_min_return'] <= sell_threshold, 'target'] = -1  # SELL
        
        # Drop forward-looking columns (data leakage prevention)
        features_df.drop(columns=['fwd_max_high', 'fwd_min_low', 'fwd_max_return', 'fwd_min_return'], inplace=True)
        
        # Drop NaN rows (first 200 bars due to long-term MAs)
        features_df.dropna(inplace=True)
        
        featured_data.append(features_df)
        print(f"   ✅ {ticker}: {len(features_df)} rows, {features_df['target'].value_counts().to_dict()}")
        
    except Exception as e:
        print(f"   ❌ {ticker}: {e}")

# Combine all featured data
full_dataset = pd.concat(featured_data, ignore_index=True)

print(f"\n{'='*60}")
print(f"✅ Feature Engineering Complete")
print(f"   Total rows: {len(full_dataset):,}")
print(f"   Features: {len(feature_engine.get_feature_names())} technical indicators")
print(f"   Target distribution: {full_dataset['target'].value_counts().to_dict()}")
print(f"   BUY class: {(full_dataset['target'] == 1).sum() / len(full_dataset) * 100:.1f}%")
print(f"   SELL class: {(full_dataset['target'] == -1).sum() / len(full_dataset) * 100:.1f}%")
print(f"   HOLD class: {(full_dataset['target'] == 0).sum() / len(full_dataset) * 100:.1f}%")
print(f"{'='*60}\n")

# Save featured data
feature_cache_path = f'{workspace}/data/alpha76_featured_data.parquet'
full_dataset.to_parquet(feature_cache_path)
print(f"✅ Featured data cached to: {feature_cache_path}")

## Step 8: Train Multi-Model Ensemble (XGBoost + RF + GB)

In [ ]:
# ==========================================
# STEP 9: TRAIN ENSEMBLE
# ==========================================

# Prepare data
X = full_dataset[feature_engine.get_feature_names()]
y = full_dataset['target']

# Convert to binary classification (BUY=1 vs SELL/HOLD=0)
# We focus on BUY precision (SELL is for risk management)
y_binary = (y == 1).astype(int)

# Train/Test split (80/20 with time-series split)
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y_binary.iloc[:split_idx], y_binary.iloc[split_idx:]

print(f"✅ Data Split:")
print(f"   Train: {len(X_train):,} rows ({y_train.sum():,} BUY signals)")
print(f"   Test:  {len(X_test):,} rows ({y_test.sum():,} BUY signals)")

# Initialize ensemble
ensemble = MultiModelEnsemble(use_gpu=True)

# Train with 3-fold time-series cross-validation
print(f"\n🏋️ Training 3-Model Ensemble...")
print(f"   Expected time: 30-60 minutes on T4 GPU")
print(f"   Models: XGBoost (GPU), Random Forest (CPU), Gradient Boosting (CPU)\n")

ensemble.train(X_train, y_train, n_splits=3)

# Evaluate on test set
print(f"\n📊 Evaluating on Test Set...")
predictions = ensemble.predict(X_test)
probabilities = ensemble.predict_proba(X_test)

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, probabilities[:, 1])

print(f"\n{'='*60}")
print(f"🏆 BASELINE RESULTS")
print(f"{'='*60}")
print(f"   Precision: {precision:.2%} (% of BUY signals that were correct)")
print(f"   Recall:    {recall:.2%} (% of real opportunities captured)")
print(f"   F1 Score:  {f1:.2%}")
print(f"   ROC-AUC:   {roc_auc:.3f}")
print(f"\n   Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print(f"{'='*60}\n")

# Save results
results = {
    'timestamp': datetime.now().isoformat(),
    'train_size': len(X_train),
    'test_size': len(X_test),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1),
    'roc_auc': float(roc_auc),
    'buy_class_ratio': float(y_train.sum() / len(y_train))
}

results_path = f'{workspace}/results/baseline_results.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"✅ Results saved to: {results_path}")

## Step 9: Save Models to Google Drive

In [ ]:
# ==========================================
# STEP 10: SAVE TRAINED MODELS
# ==========================================

# Save ensemble models
model_path = f'{workspace}/models/alpha76_ensemble_v1.pkl'
joblib.dump(ensemble, model_path)

print(f"✅ Models saved to: {model_path}")
print(f"\n📦 Download Instructions:")
print(f"   1. Go to Google Drive: {workspace}/models/")
print(f"   2. Right-click 'alpha76_ensemble_v1.pkl' → Download")
print(f"   3. Place in your local project: models/ folder")
print(f"\n🎯 Next Steps:")
print(f"   1. Share PERPLEXITY_OPTIMIZATION_BRIEF.md with Perplexity AI")
print(f"   2. Ask 10 critical questions about hyperparameter tuning")
print(f"   3. Implement recommendations in Week 1")
print(f"   4. Target: 65-70% precision (10-point improvement)")
print(f"\nIntelligence edge, not speed edge. 🚂")